In [1]:
import pandas as pd

# read in

In [2]:
df = pd.read_csv('../../2_Feature_Engineering/export/randomized_balanced_dataset.csv')
df.head()

,raw_text,severity,binary_label
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False


# clean

In [3]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [4]:
df.head()

,raw_text,severity,binary_label
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False


In [5]:
df['cleaned_text'] = df['raw_text'].apply(ci)

In [6]:
df.head()

,raw_text,severity,binary_label,cleaned_text
0,"Ich warte schon darauf, dass man die Terrorist...",0.0,False,warte schon darauf das terroristen opfern mach...
1,Holt Björn Höcke ins Boot vielleicht haben wir...,0.0,False,holt björn höcke boot vielleicht letzte chance...
2,"Deutscher, turkmenischer Abstammung bitte. Was...",0.0,False,deutscher turkmenischer abstammung bitte graue...
3,"Wer glaubt die gehen wieder , ist einfach nur ...",0.0,False,wer glaubt gehen einfach blauäugig
4,Man wünscht sich eine schlagkräftige Bürgerweh...,0.0,False,wünscht schlagkräftige bürgerwehr sophienhof p...


# vectorize

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
tfidf_vec = TfidfVectorizer(ngram_range=(2,5), analyzer='char', stop_words='german', max_features=10000)
tfidf_wb_vec = TfidfVectorizer(ngram_range=(2,5), analyzer='char_wb', stop_words='german', max_features=10000)
count_vec = CountVectorizer(ngram_range=(2,5), analyzer='char', stop_words='german', max_features=10000)
count_wb_vec = CountVectorizer(ngram_range=(2,5), analyzer='char_wb', stop_words='german', max_features=10000)

In [9]:
tfidf_features = tfidf_vec.fit_transform(df['cleaned_text'])
tfidf_wb_features = tfidf_wb_vec.fit_transform(df['cleaned_text'])
count_features = count_vec.fit_transform(df['cleaned_text'])
count_wb_features = count_wb_vec.fit_transform(df['cleaned_text'])

In [10]:
tfidf_features.shape

(1828, 10000)

In [11]:
tfidf_wb_features.shape

(1828, 10000)

In [12]:
count_features.shape

(1828, 10000)

In [13]:
count_wb_features.shape

(1828, 10000)

# main function

In [14]:
def split_train_score(features, labels, clf):
    
    # split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
    
    #train    
    clf.fit(X_train, y_train)
    
    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    
    print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                         round(recall, 3), 
                                                         round((y_pred==y_test).sum() / len(y_pred)),
                                                         round(fscore, 3)))
    # confusion matrix
    from sklearn.metrics import confusion_matrix
    matrix = confusion_matrix(y_test, y_pred)
    m_df = pd.DataFrame(
    matrix, 
    columns=["Negatives", "Positives"],
    index=["Negatives", "Positives"])
    print(m_df)
    

In [15]:
options = {'TF-IDF': tfidf_features, 'TF-IDF_with_wb': tfidf_wb_features, 'CountVectorizer': count_features, 'CountVectorizer_with_wb': count_wb_features}

# logistic regression

In [16]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], lr)
    split_train_score(option, df['binary_label'], lr)
    split_train_score(option, df['binary_label'], lr)
    print()

TF-IDF
Precision: 0.826 / Recall: 0.555 / Accuracy: 1.0 / Fscore: 0.664
           Negatives  Positives
Negatives        263         30
Positives        114        142
Precision: 0.822 / Recall: 0.565 / Accuracy: 1.0 / Fscore: 0.67
           Negatives  Positives
Negatives        265         31
Positives        110        143
Precision: 0.86 / Recall: 0.629 / Accuracy: 1.0 / Fscore: 0.726
           Negatives  Positives
Negatives        279         25
Positives         91        154

TF-IDF_with_wb


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Precision: 0.812 / Recall: 0.643 / Accuracy: 1.0 / Fscore: 0.717
           Negatives  Positives
Negatives        263         37
Positives         89        160
Precision: 0.809 / Recall: 0.613 / Accuracy: 1.0 / Fscore: 0.697
           Negatives  Positives
Negatives        265         36
Positives         96        152
Precision: 0.793 / Recall: 0.607 / Accuracy: 1.0 / Fscore: 0.688
           Negatives  Positives
Negatives        278         37
Positives         92        142

CountVectorizer
Precision: 0.69 / Recall: 0.702 / Accuracy: 1.0 / Fscore: 0.696
           Negatives  Positives
Negatives        240         74
Positives         70        165
Precision: 0.749 / Recall: 0.675 / Accuracy: 1.0 / Fscore: 0.71
           Negatives  Positives
Negatives        240         57
Positives         82        170
Precision: 0.742 / Recall: 0.657 / Accuracy: 1.0 / Fscore: 0.697
           Negatives  Positives
Negatives        248         56
Positives         84        161

CountVectorizer_wi

# Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], rf)
    split_train_score(option, df['binary_label'], rf)
    split_train_score(option, df['binary_label'], rf)
    print()

TF-IDF


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        293          0
Positives        256          0


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        307          0
Positives        242          0


...

Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        296          0
Positives        253          0


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        279          0
Positives        270          0


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        289          0
Positives        260          0

CountVectorizer
Precision: 0.491 / Recall: 0.347 / Accuracy: 1.0 / Fscore: 0.407
           Negatives  Positives
Negatives        216         88
Positives        160         85
Precision: 0.534 / Recall: 0.351 / Accuracy: 1.0 / Fscore: 0.423
           Negatives  Positives
Negatives        199         82
Positives        174         94
Precision: 0.543 / Recall: 0.369 / Accuracy: 1.0 / Fscore: 0.439
           Negatives  Positives
Negatives        215         79
Positives        161         94

CountVectorizer_with_wb
Precision: 0.915 / Recall: 0.9 / Accuracy: 1.0 / Fscore: 0.907
           Negatives  Positives
Negatives        278         21
Positives         25        225
Precision: 0.921 / Recall: 0.899 / Accuracy: 1.0 / Fscore: 0.91
           Negatives  Positives
Negatives        282         19
Positives         25        223
P

# Multinomial Naive Bayes

In [18]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], mnb)
    split_train_score(option, df['binary_label'], mnb)
    split_train_score(option, df['binary_label'], mnb)
    print()

TF-IDF
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        289          0
Positives        260          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        296          0
Positives        253          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        301          0
Positives        248          0

TF-IDF_with_wb
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        300          0
Positives        249          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        306          0
Positives        243          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        294          0
Positives        255          0

CountVectorizer
Precision: 0.539 / Recall: 0.3

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\pascal\Anaconda3\li

           Negatives  Positives
Negatives        220         79
Positives        157         93
Precision: 0.536 / Recall: 0.374 / Accuracy: 1.0 / Fscore: 0.44
           Negatives  Positives
Negatives        202         85
Positives        164         98

CountVectorizer_with_wb
Precision: 0.905 / Recall: 0.898 / Accuracy: 1.0 / Fscore: 0.901
           Negatives  Positives
Negatives        282         23
Positives         25        219
Precision: 0.924 / Recall: 0.878 / Accuracy: 1.0 / Fscore: 0.9
           Negatives  Positives
Negatives        295         17
Positives         29        208
Precision: 0.9 / Recall: 0.857 / Accuracy: 1.0 / Fscore: 0.878
           Negatives  Positives
Negatives        274         24
Positives         36        215



# Support Vector Machine

In [19]:
from sklearn.linear_model import SGDClassifier
svm = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None) # params only copied

for index, option in options.items():
    print(index)
    split_train_score(option, df['binary_label'], svm)
    split_train_score(option, df['binary_label'], svm)
    split_train_score(option, df['binary_label'], svm)
    print()

TF-IDF
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        290          0
Positives        259          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        317          0
Positives        232          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        297          0
Positives        252          0

TF-IDF_with_wb
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        305          0
Positives        244          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        296          0
Positives        253          0
Precision: 0.0 / Recall: 0.0 / Accuracy: 1.0 / Fscore: 0.0
           Negatives  Positives
Negatives        319          0
Positives        230          0

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)




CountVectorizer
Precision: 0.423 / Recall: 0.306 / Accuracy: 1.0 / Fscore: 0.355
           Negatives  Positives
Negatives        220         97
Positives        161         71
Precision: 0.528 / Recall: 0.378 / Accuracy: 1.0 / Fscore: 0.441
           Negatives  Positives
Negatives        213         85
Positives        156         95
Precision: 0.508 / Recall: 0.364 / Accuracy: 1.0 / Fscore: 0.424
           Negatives  Positives
Negatives        207         89
Positives        161         92

CountVectorizer_with_wb
Precision: 0.906 / Recall: 0.902 / Accuracy: 1.0 / Fscore: 0.904
           Negatives  Positives
Negatives        281         23
Positives         24        221
Precision: 0.907 / Recall: 0.907 / Accuracy: 1.0 / Fscore: 0.907
           Negatives  Positives
Negatives        279         23
Positives         23        224
Precision: 0.917 / Recall: 0.894 / Accuracy: 1.0 / Fscore: 0.905
           Negatives  Positives
Negatives        283         20
Positives         26   

--> option 'char_wb' on count vectorizer seems to improve the results significantly on almost every algorithm

# compare cv with char_wb

In [20]:
def split_train_score_compare(features, labels, clf):
    
    # split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)
    
    #train    
    clf.fit(X_train, y_train)
    
    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    return round(precision, 3), round(recall, 3), round((y_pred==y_test).sum() / len(y_pred)), round(fscore, 3)

In [21]:
scores = {}

In [22]:
winner_algo = []
for i in range(20):
    scores['lr'] = split_train_score_compare(count_wb_features, df['binary_label'], lr)
    scores['rf'] = split_train_score_compare(count_wb_features, df['binary_label'], rf)
    scores['mnb'] = split_train_score_compare(count_wb_features, df['binary_label'], mnb)
    scores['svm'] = split_train_score_compare(count_wb_features, df['binary_label'], svm)
    
    score_df = pd.DataFrame(list(scores.values()), index=list(scores.keys()), columns=['precision', 'recall', 'accuracy', 'fscore']).reset_index().sort_values('fscore', ascending=False)
    winner_algo.append((score_df.iloc[0]['index'], score_df.iloc[0]['fscore'], score_df.iloc[0]['accuracy']))
    print(score_df)
winner_algo

C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
1    rf      0.926   0.926       1.0   0.926
2   mnb      0.935   0.904       1.0   0.919
3   svm      0.923   0.904       1.0   0.913
0    lr      0.727   0.703       1.0   0.715


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
3   svm      0.938   0.911       1.0   0.924
2   mnb      0.924   0.920       1.0   0.922
1    rf      0.915   0.918       1.0   0.916
0    lr      0.759   0.660       1.0   0.706


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

  index  precision  recall  accuracy  fscore
2   mnb      0.939   0.935       1.0   0.937
1    rf      0.946   0.925       1.0   0.936
3   svm      0.891   0.916       1.0   0.903
0    lr      0.727   0.759       1.0   0.743


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


  index  precision  recall  accuracy  fscore
1    rf      0.942   0.938       1.0   0.940
2   mnb      0.933   0.917       1.0   0.925
3   svm      0.948   0.894       1.0   0.920
0    lr      0.771   0.686       1.0   0.726
  index  precision  recall  accuracy  fscore
3   svm      0.927   0.912       1.0   0.920
2   mnb      0.927   0.896       1.0   0.911
1    rf      0.895   0.922       1.0   0.908
0    lr      0.740   0.676       1.0   0.707


C:\Users\pascal\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


[('rf', 0.926, 1.0),
 ('svm', 0.924, 1.0),
 ('mnb', 0.929, 1.0),
 ('rf', 0.911, 1.0),
 ('rf', 0.935, 1.0),
 ('rf', 0.906, 1.0),
 ('mnb', 0.913, 1.0),
 ('mnb', 0.929, 1.0),
 ('rf', 0.926, 1.0),
 ('rf', 0.931, 1.0),
 ('svm', 0.919, 1.0),
 ('svm', 0.935, 1.0),
 ('svm', 0.938, 1.0),
 ('mnb', 0.917, 1.0),
 ('mnb', 0.943, 1.0),
 ('rf', 0.944, 1.0),
 ('mnb', 0.933, 1.0),
 ('mnb', 0.937, 1.0),
 ('rf', 0.94, 1.0),
 ('svm', 0.92, 1.0)]

In [23]:
pd.DataFrame(winner_algo, columns=['algo', 'fscore', 'accuracy']).sort_values(['algo','fscore'], ascending=[True, False])

,algo,fscore,accuracy
14,mnb,0.943,1.0
17,mnb,0.937,1.0
16,mnb,0.933,1.0
2,mnb,0.929,1.0
7,mnb,0.929,1.0
13,mnb,0.917,1.0
6,mnb,0.913,1.0
15,rf,0.944,1.0
18,rf,0.940,1.0
4,rf,0.935,1.0


# exports

In [24]:
def export_function(clf, clf_name, features, labels):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3)

    print(clf_name)
    print('-------')
    
    #train    
    clf.fit(X_train, y_train)

    # score
    from sklearn.metrics import precision_recall_fscore_support as score
    y_pred = lr.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label=True, average='binary') # pos_label= wenn nicht binary, dann label angeben
    print('Precision: {} / Recall: {} / Accuracy: {} / Fscore: {}'.format(round(precision, 3), 
                                                             round(recall, 3), 
                                                             round((y_pred==y_test).sum() / len(y_pred)),
                                                             round(fscore, 3)))
    # confusion matrix
    from sklearn.metrics import confusion_matrix
    matrix = confusion_matrix(y_test, y_pred)
    m_df = pd.DataFrame(
    matrix, 
    columns=["Negatives", "Positives"],
    index=["Negatives", "Positives"])
    print(m_df)
    
    # export
    import pickle
    pickle.dump(clf, open('exports/{}_model.pkl'.format(clf_name), 'wb'))
    
    print()

In [25]:
export_function(svm, 'svm', count_wb_features, df['binary_label'])
export_function(mnb, 'mnb', count_wb_features, df['binary_label'])
export_function(rf, 'rf', count_wb_features, df['binary_label'])

svm
-------
Precision: 0.912 / Recall: 0.9 / Accuracy: 1.0 / Fscore: 0.906
           Negatives  Positives
Negatives        298         20
Positives         23        208

mnb
-------
Precision: 0.9 / Recall: 0.885 / Accuracy: 1.0 / Fscore: 0.892
           Negatives  Positives
Negatives        282         24
Positives         28        215

rf
-------
Precision: 0.922 / Recall: 0.911 / Accuracy: 1.0 / Fscore: 0.917
           Negatives  Positives
Negatives        282         19
Positives         22        226



In [29]:
import pickle
pickle.dump(count_wb_vec, open('exports/count_wb_vectorizer.pkl', 'wb'))